---
jupyter:
  jupytext:
    text_representation:
      extension: .md
      format_name: markdown
      format_version: '1.3'
      jupytext_version: 1.16.0
  kernelspec:
    display_name: Python 3
    language: python
    name: python3
---

<!-- #region id="eb0dad00" -->
# Table des matières
1. [Une approche par petits pas](#une-approche-par-petits-pas)
1. [Lecture des données](#lecture-des-données)
1. [Recalage linéaire simple](#recalage-linéaire-simple)
1. [Affichage des résultats](#affichage-des-résultats)
1. [Interprétation des résultats](#interprétation-des-résultats)
1. [Comparaison des profits réels et prédits](#comparaison-des-profits-réels-et-prédits)

# Attention!
Ne lancez pas l'exécution automatique du notebook en entier en cliquant sur le bouton **Tout exécuter**. L'exécution serait interrompue, car certaines cellules exigent une entrée de votre part!

Il faut simplement exécuter le notebook, une cellule à la fois, et entrer quelques lignes de code lorsque demandées. Il est inutile de sauter ces cellules pour aller aux suivantes car celles-ci ont justement besoin de votre input!

Importons d'abord les librairies nécessaires.
<!-- #endregion -->



In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

# semence pour le générateur de nombres pseudo-aléatoires
# permet d'assurer la reproductibilité des résultats
seed = 42

# initialisation du générateur à partir de la semence
np.random.seed(seed)



<!-- #region id="7a046863" -->
Dans cet exemple, nous allons montrer un autre exemple d'application des modèles de régression décrits
dans la section sur la théorie de la régression.

Nous allons utiliser le jeu de données [*startups data*](https://www.kaggle.com/amineoumous/50-startups-data) provenant du site de Kaggle.

Elle liste les profits de 50 compagnies jeunes-pousses (*startups*) uniformément réparties dans
les états de New York, de la Californie et de la Floride. Elle contient les variables suivantes:

- `R&D Spend`: dépenses investies en recherche et développement.
- `Administration`: frais d'administration.
- `Marketing Spend`: dépenses investies en marketing.
- `State`: localisation géographique (New York, California, Florida).

Les trois premières sont continues alors que la quatrième est catégorielle. On voudrait prédire les profits
des compagnies en fonction de ces variables.
<!-- #endregion -->

<!-- #region id="89a352e8" -->
## <span id=une-approche-par-petits-pas>Une approche par petits pas</span>

<!-- #endregion -->

<!-- #region id="a82f52a6" -->
Puisqu'il n'existe pas de modèle analytique exact dans ce type de problème, nous allons utiliser
des modèles linéaires pour analyser les données. Bien que très simples, ils se sont souvent révélés utiles
pour analyser des données financières. De plus, leur simplicité facilite l'interprétation des résultats et
l'identification des variables les plus importantes.

Dans cet exemple, nous allons utiliser un modèle linéaire sans variable catégorique, soit uniquement les variables continues.

Il faut savoir qu'une grande part des problèmes en apprentissage automatique utilisent des données de différents types; numériques, ordinales et catégorielles. Il faut apprendre à les utiliser correctement. Ce point important est discuté dans le module sur le prétraitement des données. Nous allons en donner un exemple simple d'utilisation dans ce qui suit.

Notre modèle correspond donc à :
$$\text{Profit} \approx a_{0} + a_{1}\text{R&D} + a_{2}\text{Administration} + a_{3}\text{Marketing}.$$
<!-- #endregion -->

<!-- #region id="da134e55" -->
## <span id=lecture-des-données>Lecture des données</span>
<!-- #endregion -->

<!-- #region id="6ba7b8ab" -->
La dernière colonne correspond aux profits des compagnies (en $\text{k}~\$$). C'est la réponse $y$ que l'on veut prédire à partir des variables $X$ dans les colonnes précédentes.
<!-- #endregion -->



In [ ]:
# Lecture du jeu de données en format CSV.
dataset = pd.read_csv("/pax/shared/GIF-U014/50_Startups.csv")

# Affichage des cinq premières lignes du fichier.
dataset.head()



<!-- #region id="f8143126" -->
## <span id=recalage-linéaire-simple>Recalage linéaire simple</span>
<!-- #endregion -->

<!-- #region id="900ef68e" -->
Débutons par séparer les variables X et la réponse y du jeu de données en gardant seulement les trois premières colonnes.
<!-- #endregion -->



In [ ]:
X = dataset.iloc[:, :-1]
y = dataset.iloc[:, 4]

# Élimination de l'information géographique
X = X.drop("State", axis=1)



<!-- #region id="0828cc7e" -->
Par la suite, séparons les données en un ensemble d'entraînement ($80~\%$ des données) et de test ($20~\%$ des données).

Les données d'entraînement vont servir à entraîner le modèle de régression. C'est-à-dire, à estimer les valeurs des paramètres $a_0, a_1, a_2, a_3$. Les données de test vont ensuite servir à mesurer les performances du modèle sur des données jamais utilisées.
<!-- #endregion -->



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)



<!-- #region id="4d859e5c" -->
Par la suite, nous allons normaliser les données.

La normalisation a pour effet de mettre toutes les variables $x_i$ sur un même pied d'égalité. Les différences d'unités de mesure et d'échelle de grandeur sont éliminées. La normalisation est expliquée plus en détail dans un des modules sur le prétraitement des données.

Il faut sélectionner une fonction de normalisation et l'entraînement avec les données d'entraînement. On peut ensuite l'appliquer aux données de test.

Nous allons utiliser la normalisation centrée réduire en utilisant le [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) de Scikit-learn, chaque colonne $x_i$ de la matrice $X$ est transformée de la façon suivante

$$x'_i = \dfrac{x_i-\mu_i}{\sigma_i}$$

où $\mu_i$ et $\sigma_i$ sont la moyenne et l'écart-type des valeurs de $x_i$ calculées avec les données d'entraînement.

Dans la cellule ci-dessous, complétez les énoncés pour créer l'objet de normalisation (type `StandardScaler`), pour ajuster ses paramètres (`fit`) sur les données d'entraînement et pour appliquer (`transform`) cette transformation sur les données de test.
<!-- #endregion -->



In [ ]:
# créer un outil de transformation
norm = # à compléter

# ajuster les paramètres de l'outil sur les données d'entraînement
X_train = # à compléter

# appliquer l'outil sur les données de test
X_test = # à compléter



<!-- #region id="b08f4813" -->
Finalement, entrainons le modèle de [régression linéaire](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) par défaut de Scikit-learn et calculons la métrique $R^2$ à partir des vraies valeurs et les valeurs prédites pour les données de test. Pour cela, complétez le code ci-dessous&puncsp;:
<!-- #endregion -->



In [ ]:
# créer le modèle de regression
reg = # à compléter

# ajuster le modèle sur les données d'entraînement
reg # à compléter

# prédire sur les données de test
y_pred = # à compléter

# calculer la métrique R2
score = # à compléter



<!-- #region id="401d583a" -->
## <span id=affichage-des-résultats>Affichage des résultats</span>

Ci-dessous le code qui permet d'afficher les paramètres de modélisation&puncsp;:
<!-- #endregion -->



In [ ]:
print("Coefficients:\n")
print(f"\t a_{0} = {reg.intercept_}")
for i, coeff in enumerate(reg.coef_):
    print(f"\t a_{i+1} = {reg.coef_[i]}")

print(f"\nMétrique R2: {100*score:0.1f} %")



<!-- #region id="be2eb45d" -->
### <span id=interprétation-des-résultats>Interprétation des résultats</span>
<!-- #endregion -->

<!-- #region id="6a975318" -->
Selon la métrique $R^2$, vous devriez obtenir que ce premier modèle explique $90,0~\%$ de l'information dans les données.

Vous devriez observer que $|a_1| > |a_3| > |a_2|$, que $a_1$ et $a_3$ sont positifs et que $a_2$ est négatif.

Ainsi, les variables en ordre d'importance sont `R&D Spend`, `Marketing Spend` et `Administration`.

Plus les dépenses en R&D et marketing seront importantes, meilleurs seront les profits. Celles en R&D sont les plus importantes. Celles en administration contribuent négativement, même s'il importe sans doute d'engager aussi des personnes pour gérer le personnel en R&D et marketing.

Attention toutefois, on observe des corrélations entre les variables, pas nécessairement des liens de causalité! En effet, est-ce qu'une compagnie faisant plus de profits investit plus en R&D ou est-ce que ce sont plutôt les investissements en R&D qui mènent à plus de profits? Ce modèle ne peut pas répondre à cette question.

Autre point important. Il est possible que le jeu de données ne contienne que des données provenant de compagnies ayant survécu à leurs premières années d'existence. Il y a donc un effet de sélection et les conclusions qu'on pourrait être tenté de tirer peuvent être biaisées. Elles ne s'appliqueraient alors qu'aux compagnies ayant survécu à ces premières années d'existence!
<!-- #endregion -->

<!-- #region id="b5dbdb99" -->
### <span id=comparaison-des-profits-réels-et-prédits>Comparaison des profits réels et prédits</span>
<!-- #endregion -->



In [ ]:
# Conversion en k$
y_test = y_test / 1000
y_pred = y_pred / 1000

# utiliser les couleurs par défaut de seaborn
sns.set(color_codes=True)

# tracer le graphique des profits prédits versus les réels
fig = plt.figure(figsize=(8, 6))
plt.plot(y_test, y_pred, "o")
plt.plot(y_test, y_test, "-r", label="Prédiction parfaite")
plt.title("Modèle n'utilisant pas l'information géographique", fontsize=16)
plt.xlabel("Profit réel (k$)", fontsize=16)
plt.ylabel("Profit prédit (k$)", fontsize=16)
plt.legend()
fig.tight_layout()
plt.show()



<!-- #region id="b5934bf3" -->
Bien que le modèle linéaire utilisé soit simple, ses prédictions sont plutôt bonnes comme le montre l'adéquation entre les
valeurs prédites et les valeurs réelles de test. Les points sont bien répartis le long de la droite de prédiction parfaite.
<!-- #endregion -->
